In [ ]:
import sys
#1. DPChange
def Change(money: int, Coins: List[int]) -> int:
    # build an array of length(money) to store the minimum number of coins needed for specific money
    # set the MinNumCoins as positive infinite, set the base case
    MinNumCoins = [float('inf')]*(money+1)
    MinNumCoins[0] = 0
    for m in range(1, money+1):
        # traverse every possible coin value
        for i in range(len(Coins)):
            if m >= Coins[i]:
                if MinNumCoins[m-Coins[i]] + 1 < MinNumCoins[m]:
                    MinNumCoins[m] = MinNumCoins[m-Coins[i]] + 1
    return MinNumCoins[money]


#2. LongestCommonPath
def LongestPathLength(n: int, m: int,
                      Down: List[List[int]], Right: List[List[int]]) -> int:
    # the (n+1)*(m+1) matrix will store the maximum state of every nodes.
    state = [[0 for _ in range(m+1)] for _ in range(n+1)]
    
    # for each row
    for i in range(1, n+1):
        state[i][0] = state[i-1][0] + Down[i-1][0]
    # for each column
    for j in range(1, m+1):
        state[0][j] = state[0][j-1] + Right[0][j-1]
    
    # traverse every node that is not in the first row and first column
    for i in range(1, n+1):
        for j in range(1, m+1):
            state[i][j] = max(state[i-1][j]+Down[i-1][j], state[i][j-1]+Right[i][j-1])
    return state[n][m]


#3. OutPutLCS
sys.setrecursionlimit(10000)
# Insert your LongestCommonSubsequence function here, along with any subroutines you need
def BackTrack(s, t):
    state = [[0 for _ in range(len(s)+1)]for _ in range(len(t)+1)]
    backtrack = [[None for _ in range(len(s)+1)]for _ in range(len(t)+1)]
    # each column
    for i in range(len(s)+1):
        state[0][i] = 0
    # each row
    for j in range(len(t)+1):
        state[j][0] = 0
    
    # traverse every possible match
    for i in range(1, len(s)+1):
        for j in range(1, len(t)+1):
            match = 0
            # if the nucleotide matches
            if s[i-1] == t[j-1]:
                match = 1
            # either match or mismatch or gap (corresponding to diagonal, down, right)
            state[j][i] = max(state[j-1][i-1]+match, state[j-1][i], state[j][i-1])
            
            if state[j][i] == state[j-1][i-1]+match:
                backtrack[j][i] = "match"
            elif state[j][i] == state[j][i-1]:
                backtrack[j][i] = "right"
            elif state[j][i] == state[j-1][i]:
                backtrack[j][i] = "down"
    return backtrack
    
def LongestCommonSubsequence(s: str, t: str) -> str:
    backtrack = BackTrack(s,t)
    i = len(s)
    j = len(t)
    # do the recursion
    LCS = OutPutLCS(backtrack, s, t, i, j)
    return LCS
    
    
def OutPutLCS(backtrack, s, t, i, j):
    if i == 0 or j == 0:
        return ""
    if backtrack[j][i] == "match":
	# notice that the backtrack has additional row 0 and column 0, so the index should be adjusted.
        if s[i-1] == t[j-1]:
            return OutPutLCS(backtrack, s, t, i-1, j-1) + s[i-1]
        else:
            return OutPutLCS(backtrack, s, t, i-1, j-1)
    elif backtrack[j][i] == "right":
        return OutPutLCS(backtrack, s, t, i-1, j)
    else:
        return OutPutLCS(backtrack, s, t, i, j-1)



#4. DAG LongestPath
sys.setrecursionlimit(10000)
# Insert your LongestPath function here, along with any subroutines you need
# s and t are the starting and the ending nodes of the path respectively
# E[u] is the list of neighbors of the vertex u, paired with corresponding edge weights
# LongestPath should return the length of the longest path betweeen s and t together with
# the list of nodes of the path
def LongestPath(s: int, t: int,
                E: Dict[int, List[Tuple[int, int]]]) -> Tuple[int, List[int]]:
    
    # find all possible nodes that is reachable to the start point
    node_indices = dfs(E, s)
    # state stores the longest path of every point so far and the connected predecessor
    state = {node:0 for node in node_indices}
    # every point should have a predecessor except for the start point
    # predecessor[s] = predecessor of point s
    predecessor = {node:None for node in node_indices}
    
    # traverse every connected point between s and t    
    for i in node_indices:
        if i == t:
            continue
        if i not in E.keys():
            continue
        # traverse every neighbor of the current point
        # update the successor's state and predecessor list if there is a longer path
        for edge in E[i]:# these are current point i's connected edges
            # neighbors of current Point i
            successor = edge[0]
            if successor not in node_indices:
                continue
            weight = edge[1]
            if state[successor] < state[i] + weight:
                state[successor] = state[i] + weight
                predecessor[successor] = i
    
    path = [t]
    path = findPath(predecessor, s, t, path)
    return (state[t],path)
    
def findPath(predecessor, s , pointIndex, path):
    if pointIndex == s:
        # reverse the list 
        return reversed(path)
    else:
        parent = predecessor[pointIndex]
        path.append(parent)
        return findPath(predecessor,s, parent, path)
        

def dfs(E, start, visited=None):
    if visited is None:
        visited = set()
    visited.add(start)
    if start not in E.keys():
        return visited
    for edge in E[start]:
        neighbor = edge[0]
        if neighbor not in visited:
            dfs(E, neighbor, visited)
    return visited
